# Sep.4th Meeting 

## One existing bug

If we are doing finite difference in x direction:
if mod(Nx,TILE_DIM_X) = 1: bug even HALO_WIDTH >= 2
if mod(Nx,TIME_DIM_X) >= 2: looks okay


One way to avoid this: set TILE_DIM according to the value of Nx to avoid this

Nx = 5

TILE_DIM = 2, 4 : bug
but TILE_DIM = 3: no bug  mod(5,3) == 2



Nx = Ny = 1001

D2y

TILE_DIM = (4,16)
 
TILE_DIM = (16,4)
